#  Определение предельных динамических характеристик позиционера 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Максимальное ускорение

$$
\varepsilon_{max} = \frac{N_{max} - N_{тр}}{J} = \frac{k_M \cdot I_{max} - N_{тр}}{J}
$$

* $\varepsilon_{max}$ - максимальное угловое ускорение
* $N_{тр}$ - момент силы сухого трения
* $k_M$ - коэффициент момента - коэффициент пересчёта тока в момент силы \[Н*м / А\], берётся из документации на двигатель
* $I_{max}$ - максимальный среднеквадратичный ток
* $J$ - момент инерции ротора относительно оси вращения

### Максимальный рывок

$$
U_{max} = L\left(\frac{dI}{dt}\right)_{max}
$$

$$
\zeta_{max} = \left(\frac{d\varepsilon}{dt}\right)_{max} = \frac{k_M}{J} \left(\frac{dI}{dt}\right)_{max} = \frac{k_M}{J} \frac{U_{max}}{L}
$$

* $L$ - индуктивность обмоток двигателя, берётся из документации на двигатель
* $U_{max}$ - максимальное напряжение
* $\zeta_{max}$ - максимальный угловой рывок (производная от углового ускорения)

### Ограничение на скорость из учёта обратной индукции

Вводим требование:

$$
\frac{U_{EMF}}{U_{max}} < 30 \%
$$

$$
\omega < \frac{3}{10} \frac{U_{max}}{k_{EMF}}
$$

* $U_{EMF}$ - обратная ЭДС
* $U_{MAX}$ - максимальное напряжение
* $k_{EMF}$ - коэффициент обратной ЭДС (противоЭДС) \[В / (тыс. об/мин)\], берётся из документации на двигатель
* $\omega$ - угловая скорость \[тыс. об/мин\]

## Калькулятор предельных динамических характеристик

In [ ]:
layout = widgets.Layout(width='400px')
style = {'description_width': '140px'}
n_tr_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=0,
    min=0,
    max=100,
    step=0.1,
    description="N_тр, Н*м",
    tooltip="Момент силы сухого трения в Ньютонах * метр"
)
k_m_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=12.9,
    min=0.001,
    max=100000,
    step=0.1,
    description="k_m, Н*м/А",
    tooltip="Коэффициент момента (берётся из документации на двигатель)"
)
i_max_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=1,
    min=0.01,
    max=100,
    step=0.1,
    description="I_max, A",
    tooltip="Максимальный ток"
)
j_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=2.2,
    min=0.0001,
    max=1000,
    step=0.1,
    description="J, кг*м^2",
    tooltip="Момент инерции ротора относително оси вращения"
)
u_max_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=36,
    min=0.01,
    max=1000,
    step=0.1,
    description="U_max, В",
    tooltip="Максимальное напряжение"
)
inductance_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=0.00124,
    min=0.0000001,
    max=0.1,
    step=0.1,
    description="L, Гн",
    tooltip="Индуктивность обмоток двигателя, берётся из документации на двигатель"
)
k_emf_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=15,
    min=0.0000001,
    max=100000,
    step=0.1,
    description="k_emf, В/(тыс. об/мин)",
    tooltip="Коэффициент обратной ЭДС, берётся из документации на двигатель"
)

calculate_dynamic_limits_button = widgets.Button(
    description="Рассчитать"
)

dl_output = widgets.Output()

def calculate_dynamic_limits(b):
    with dl_output:  
        dl_output.clear_output()
        n_tr = float(n_tr_input.value)
        k_m = float(k_m_input.value)
        i_max = float(i_max_input.value)
        j = float(j_input.value)
        u_max = float(u_max_input.value)
        inductance = float(inductance_input.value)
        k_emf = float(k_emf_input.value)
        
        beta_max = (k_m * i_max - n_tr) / j
        j_max = k_m * u_max / (j * inductance)
        v_emf_max = 0.3 * u_max / k_emf
        
        print("Максимальное ускорение: {} рад/с²".format(np.round(beta_max, 3)))
        print("Максимальный рывок: {} рад/с³".format(np.round(j_max, 0)))
        print("Ограничение на скорость из учёта обратной индукции: {} тыс. об/мин".format(np.round(v_emf_max, 6)))

calculate_dynamic_limits_button.on_click(calculate_dynamic_limits)

display(
    n_tr_input,
    k_m_input,
    i_max_input,
    j_input,
    u_max_input,
    inductance_input,
    k_emf_input,
    calculate_dynamic_limits_button,
    dl_output
)

BoundedFloatText(value=0.0, description='N_тр, Н*м', layout=Layout(width='400px'), step=0.1, style=Description…

BoundedFloatText(value=12.9, description='k_m, Н*м/А', layout=Layout(width='400px'), max=100000.0, min=0.001, …

BoundedFloatText(value=1.0, description='I_max, A', layout=Layout(width='400px'), min=0.01, step=0.1, style=De…

BoundedFloatText(value=2.2, description='J, кг*м^2', layout=Layout(width='400px'), max=1000.0, min=0.0001, ste…

BoundedFloatText(value=36.0, description='U_max, В', layout=Layout(width='400px'), max=1000.0, min=0.01, step=…

BoundedFloatText(value=0.00124, description='L, Гн', layout=Layout(width='400px'), max=0.1, min=1e-07, step=0.…

BoundedFloatText(value=15.0, description='k_emf, В/(тыс. об/мин)', layout=Layout(width='400px'), max=100000.0,…

Button(description='Рассчитать', style=ButtonStyle())

Output()

## Подбор оптимальной тестовой трапеции

$$
\alpha = \alpha_0 + \omega_0 t + \varepsilon_0\frac{t^2}{2} + \zeta\frac{t^3}{6} \\
\omega = \omega_0 + \varepsilon_0 t + \zeta\frac{t^2}{2} \\
\varepsilon = \varepsilon_0 + \zeta t
$$

Перемещение на расстояние $\alpha_{target}$ в общем случае включает в себя следующие фазы:

* Рывок (увеличение / уменьшение ускорения) – 4 сегмента
* Равноускоренное / равнозамедленное движение – 2 сегмента
* Движение с постоянной скоростью – 1 сегмент

### Рывок (набор ускорения)

Движение в любом случае начинается с рывка.

$$
\varepsilon(t) = \zeta t \\
\omega(t) = \zeta \frac{t^2}{2}\\
\alpha(t) = \zeta \frac{t^3}{6}
$$

Рывок заканчивается в тот момент, когда наступает первое (по времени) из следующих событий:
* Достигнуто максимальное ускорение $\varepsilon(t_{jerk}) = \varepsilon_{max}$ – ускоряться дальше нельзя.
* Скорость стала равна половине максимальной $\omega(t_{jerk}) = \frac{1}{2}\omega_{max}$ – пора делать обратный рывок, чтобы прекратить ускорение к достижению $\omega=\omega_{max}$.
* Пройдена четверть пути $\alpha(t_{jerk}) = \frac{1}{12}\alpha_{target}$ – пора уменьшать ускорение и сразу начинать торможение, иначе не успеем затормозить до достижения $\alpha = \alpha_{target}$.


$$
t_{jerk} = min\left(\frac{\varepsilon_{max}}{\zeta} ;\quad \sqrt{\frac{\omega_{max}}{\zeta}};\quad \sqrt[3]{\frac{\alpha_{max}}{2\zeta}} \right) \\
$$

> #### Почему в третьем условии $\frac{1}{12}\alpha_{target}$?
>
> Рассмотрим перемещение состоящее только из двух положительных рывков и двух отрицательных рывков. 
> Точнее только из соображений симметрии будем рассматривать только половину за которую мы должны пройти $\frac{1}{2}\alpha_{target}$.
>
> Положительный рывок уже рассмотрен выше. Смотрим координату после отрицательного рывка.
>
> После отрицательного рывка ускорение должно быть равно нулю: 
 
$$
0 = \varepsilon_{jerk} - \zeta t_{jerk_2} \Rightarrow t_{jerk_2} = \frac{\varepsilon_{jerk}}{\zeta} = \frac{\zeta t_{jerk_1}}{\zeta} = t_{jerk_1} = t_{jerk}
$$

> Пройденное расстояние после отрицательного рывка должно быть $\frac{\alpha_{target}}{2}$:

$$
\frac{1}{2}\alpha_{target} = \alpha_{jerk} + \omega_{jerk} t_{jerk} + \varepsilon_{jerk} \frac{t^2_{jerk}}{2} - \zeta \frac{t^3_{jerk}}{6} = 
\zeta \frac{t^3_{jerk}}{6} + \zeta \frac{t^3_{jerk}}{2} + \zeta \frac{t^3_{jerk}}{2} - \zeta\frac{t^3_{jerk}}{6} = \zeta t^3_{jerk} \\
t_{jerk} = \sqrt[3]{\frac{\alpha_{target}}{2\zeta}} \\
\alpha_{jerk} = \frac{\alpha_{target}}{12}
$$

### Равноускоренное движение

Если после первого рывка обратный рывок не начинается сразу, то далее следует сегмент равноускоренного движения.

На входе:

$$
\varepsilon_{jerk} = \zeta t_{jerk}\\
\omega_{jerk} = \zeta \frac{t^2_{jerk}}{2}\\
\alpha_{jerk} = \zeta \frac{t^3_{jerk}}{6}
$$

В процессе движения:

$$
\varepsilon(t) = \varepsilon_{jerk} = \zeta t_{jerk}\\
\omega(t) = \omega_{jerk} + \varepsilon_{jerk} t = \zeta t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right)\\
\alpha(t) = \alpha_{jerk} + \omega_{jerk} t + \varepsilon_{jerk} \frac{t^2}{2} = \zeta t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
$$


Равноускоренное движение завершается при возникновении первого из следующих событий:
* Достигнута скорость $\omega(t) = \omega_{max} – 2\omega_{jerk}$ - пора начинать завершать ускорение, иначе скорость может превысить $\omega_{max}$.
* Пройден такой путь  $\alpha(t)$, что сразу после завершения обратного рывка, начатого в данной точке, будет пройден путь $\frac{1}{2}\alpha_{target}$.


$$
t_{\alpha} = min\left(t_{jerk}\left(\frac{\omega_{max}}{\zeta t^2_{jerk}} - 1 \right);\;  t_{jerk} \frac{1}{2}\left(\sqrt{1 + 4\frac{\alpha_{target}}{\zeta t^3_{jerk}}} - 3 \right)\right)
$$

> #### Как получилось критическое время для достижения максимальной скорости

$$
\omega_{max} - \omega_{jerk} = \omega_{jerk} + \varepsilon_{jerk}t \\
t = \frac{\omega_{max}}{\varepsilon_{jerk}} - 2\frac{\omega_{jerk}}{\varepsilon_{jerk}} = \frac{\omega_{max}}{\zeta t_{jerk}} - 2\frac{\frac{1}{2}\zeta t^2_{jerk}}{\zeta t_{jerk}} = t_{jerk}\left( \frac{\omega_{max}}{\zeta t^2_{jerk}} - 1\right)
$$

> #### Как получилось критическое время для прохождения $\frac{1}{2}\alpha_{target}$
>
> После завершения участка равноускоренного движения:
 
$$
\omega(t) = \zeta t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right) \\
\alpha(t) = \zeta t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
$$

> Далее следует отрицательный рывок, в ходе которого ускорение $\varepsilon_{jerk}$ должно уменьшиться до нуля: $0 = \varepsilon_{jerk} - \zeta t_{jerk_2}$. Следовательно: $t_{jerk_2} = \frac{\varepsilon_{jerk}}{\zeta} = t_{jerk_1} = t_{jerk}$.
>
> Пройденное расстояние после этого рывка должно быть $\frac{1}{2}\alpha_{target}$:

$$
\frac{1}{2}\alpha_{target} = \alpha_\varepsilon + \omega_\varepsilon t_{jerk} + \varepsilon_{jerk}\frac{t_{jerk}}{2} - \zeta\frac{t^3_{jerk}}{6} = \\
= \zeta t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
+ \zeta t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right) t_{jerk} + \zeta t_{jerk}\frac{t_{jerk}}{2} - \zeta\frac{t^3_{jerk}}{6} = \\
= \zeta t^3_{jerk}\frac{1}{2}  \left(\left(\frac{t}{t_{jerk}}\right)^2 + 3\frac{t}{t_{jerk}} + 2 \right)
$$

> Время равноускоренного движение $t_\varepsilon$ является решением квадратного уравнения:

$$
\zeta t^3_{jerk}\frac{1}{2}  \left(\left(\frac{t}{t_{jerk}}\right)^2 + 3\frac{t}{t_{jerk}} + 2  - \frac{\alpha_{target}}{\zeta t^3_{jerk}}\right) = 0\\
t = t_{jerk}\frac{1}{2} \left(\sqrt{1 + 4\frac{\alpha_{target}}{\zeta t^3_{jerk}}} -3\right)
$$

### Отрицательный рывок

Длительность отрицательного рывка равна длительности первого рывка $t_{jerk}$, которая была посчитана ранее (доказано выше).

На входе:

$$
\varepsilon_{target} = \varepsilon_{jerk} = \zeta t_{jerk}\\
\omega_\varepsilon = \omega_{jerk} + \varepsilon_{jerk} t_\varepsilon = \zeta t^2_{jerk} \left(\frac{1}{2} + \frac{t_\varepsilon}{t_{jerk}} \right)\\
\alpha_\varepsilon = \alpha_{jerk} + \omega_{jerk} t_\varepsilon + \varepsilon_{jerk} \frac{t_\varepsilon^2}{2} = \zeta t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t_a}{t_{jerk}} + \frac{1}{2}\left(\frac{t_\varepsilon}{t_{jerk}}\right)^2\right)
$$

В процессе движения:

$$
\varepsilon(t) = \varepsilon_{jerk} - \zeta t = \zeta(t_{jerk} - t)\\
\omega(t) = \omega_\varepsilon + \varepsilon_{jerk} t - \zeta \frac{t^2}{2} = \zeta t^2_{jerk} \left( \frac{1}{2} + \frac{t_\varepsilon}{t_{jerk}} + \frac{t}{t_{jerk}} - \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2 \right) \\
\alpha(t) = \alpha_\varepsilon + \omega_\varepsilon t + \varepsilon_{jerk} \frac{t^2}{2} - \zeta\frac{t^3}{6} = \zeta t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t_\varepsilon}{t_{jerk}} + \frac{1}{2}\left(\frac{t_\varepsilon}{t_{jerk}}\right)^2 + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{t_\varepsilon}{t_{jerk}}\frac{t}{t_{jerk}} + \frac{1}{2} \left(\frac{t}{t_{jerk}}\right)^2 - \frac{1}{6} \left(\frac{t}{t_{jerk}}\right)^3 \right)
$$

### Равномерное движение

На входе:

$$
\varepsilon = 0\\
\omega_{target} = \zeta t^2_{jerk} \left( \frac{t_\varepsilon}{t_{jerk}} + 1 \right) \\
\alpha_{speedup} = \zeta t^3_{jerk} \frac{1}{2} \left( \left(\frac{t_\varepsilon}{t_{jerk}}\right)^2 + 3\frac{t_\varepsilon}{t_{jerk}} +2  \right)
$$

В процессе движения:

$$
\varepsilon = 0\\
\omega = \omega_{target} = \zeta t^2_{jerk} \left( \frac{t_\varepsilon}{t_{jerk}} + 1 \right)\\
\alpha(t) = \alpha_{speedup} + \omega_{target}t = \zeta t^3_{jerk} \frac{1}{2} \left( \left(\frac{t_\varepsilon}{t_{jerk}}\right)^2 + 3\frac{t_\varepsilon}{t_{jerk}} +2 + \frac{t_\varepsilon}{t_{jerk}}\frac{t}{t_{jerk}} + \frac{t}{t_{jerk}} \right) = \zeta t^3_{jerk} \frac{1}{2} \left( \frac{t_\varepsilon}{t_{jerk}} + 1 \right) \left( \frac{t_\varepsilon}{t_{jerk}} + 2 \frac{t}{t_{jerk}} \right)
$$

Полный разгон и полное торможение занимают одно и то же время $t_{\varepsilon} + 2t_{jerk}$ и одно и то же расстояние $\alpha_{speedup}$. Соответственно, расстояние, которое будет пройдено с фиксированной скоростью: $\alpha_{\omega} = \alpha_{target} – 2 \alpha_{speedup}$. Зная это расстояние, можно вычислить время равномерного движения $t_\omega$.

$$
t_{\omega} = \frac{\alpha_{target} - 2\alpha_{speedup}}{\omega_{target}} = \frac{\alpha_{target}}{\zeta t^2_{jerk}\left( \frac{t_\varepsilon}{t_{jerk}} + 1\right)} - \left( t_\varepsilon + 2 t_{jerk}\right)
$$

## Основные формулы ещё раз

$$
t_{jerk} = min\left(\frac{\varepsilon_{max}}{\zeta} ;\quad \sqrt{\frac{\omega_{max}}{\zeta}};\quad \sqrt[3]{\frac{\alpha_{max}}{2\zeta}} \right) \\
t_\varepsilon = min\left(t_{jerk}\left(\frac{\omega_{max}}{\zeta t^2_{jerk}} - 1 \right);\;  t_{jerk} \frac{1}{2}\left(\sqrt{1 + 4\frac{\alpha_{target}}{\zeta t^3_{jerk}}} - 3 \right)\right) \\
t_{\omega} = \frac{\alpha_{target}}{\zeta t^2_{jerk}\left( \frac{t_\varepsilon}{t_{jerk}} + 1\right)} - \left( t_\varepsilon + 2 t_{jerk}\right) \\
$$


$$
\varepsilon_{target} = \zeta t_{jerk}\\
\omega_{target} = \zeta t^2_{jerk} \left( \frac{t_\varepsilon}{t_{jerk}} + 1 \right)
$$

In [46]:
def calcaulate_timings_and_targets(alfa_target, omega_max, eps_max, zeta, angle_unit='radian'):
    t_jerk = np.min((eps_max / zeta, np.sqrt(omega_max/zeta), np.cbrt(alfa_target / (2*zeta))))

    t_eps = np.min((
        t_jerk * (omega_max / (zeta * t_jerk * t_jerk) - 1),
        t_jerk * 0.5 * (np.sqrt(1 + 4 * alfa_target / (zeta * t_jerk * t_jerk * t_jerk)) - 3)
    ))
    t_omega = alfa_target / (zeta * t_jerk * t_jerk * (t_eps / t_jerk + 1)) - (t_eps + 2 * t_jerk)

    eps_target = zeta * t_jerk
    omega_target = zeta * t_jerk * t_jerk * (t_eps / t_jerk + 1)

    print("Длительность рывка t_jerk: {} с".format(t_jerk))
    print("Длительность равноускоренного движения t_ε: {} с".format(t_eps))
    print("Длительность равномерного движения t_ω: {} с".format(t_omega))
    if angle_unit == 'radian':
        print("Реальное максимальное ускорение ε_target: {} рад/с²".format(eps_target))
        print("Реальная максимальная скорость ω_target: {} рад/с".format(omega_target))
    elif angle_unit == 'degree':
        print("Реальное максимальное ускорение ε_target: {} град/с²".format((180/np.pi*eps_target)))
        print("Реальная максимальная скорость ω_target: {} град/с".format(180/np.pi*omega_target))
    elif angle_unit == 'revolution':
        print("Реальное максимальное ускорение ε_target: {} об/мин²".format((3600*eps_target/(2*np.pi))))
        print("Реальная максимальная скорость ω_target: {} об/мин".format(60*omega_target/(2*np.pi)))
    
    return(t_jerk, t_eps, t_omega, eps_target, omega_target)

In [47]:
def calculate_time_series(t_jerk, t_eps, t_omega, zeta):
    h_t = 1 / 10000000  # c

    t_segments = [
        0,                                                             # 0 - positive jerk
        t_jerk,                                                        # 1 - positive acceleration
        t_jerk + t_eps,                                                # 2 - negative jerk
        t_jerk + t_eps + t_jerk,                                       # 3 - uniform motion
        t_jerk + t_eps + t_jerk + t_omega,                             # 4 - negative jerk
        t_jerk + t_eps + t_jerk + t_omega + t_jerk,                    # 5 - negative acceleration
        t_jerk + t_eps + t_jerk + t_omega + t_jerk + t_eps,            # 6 - positive jerk
        t_jerk + t_eps + t_jerk + t_omega + t_jerk + t_eps + t_jerk,   # 7 - the end
    ]

    t_segment_idxs = []
    for ts in t_segments:
        t_segment_idxs.append(int(ts / h_t))

    t_series = np.linspace(0, t_segments[-1], t_segment_idxs[-1])
    zeta_series = np.zeros(len(t_series))

    zeta_series[t_segment_idxs[0]:t_segment_idxs[1]] = zeta
    zeta_series[t_segment_idxs[2]:t_segment_idxs[3]] = -zeta
    zeta_series[t_segment_idxs[4]:t_segment_idxs[5]] = -zeta
    zeta_series[t_segment_idxs[6]:t_segment_idxs[7]] = zeta

    eps_series = np.cumsum(zeta_series) * h_t
    omega_series = np.cumsum(eps_series) * h_t
    alfa_series = np.cumsum(omega_series) * h_t
    
    return (t_series, zeta_series, eps_series, omega_series, alfa_series)

In [62]:
def plot_trapezoid(
    t_series, zeta_series, eps_series, omega_series, alfa_series,
    eps_target, omega_target, alfa_target, omega_max, eps_max,
    angle_unit='radian'
):
    plot_step = 1
    if len(t_series) > 10000:
        plot_step = len(t_series) // 10000

    if angle_unit == 'degree':
        alfa_series_loc = alfa_series*180/np.pi
        omega_series_loc = omega_series*180/np.pi
        eps_series_loc = eps_series*180/np.pi
        zeta_series_loc = zeta_series*180/np.pi
        alfa_target_loc = alfa_target*180/np.pi
        omega_max_loc = omega_max*180/np.pi
        omega_target_loc = omega_target*180/np.pi
        eps_max_loc = eps_max*180/np.pi
        eps_target_loc = eps_target*180/np.pi
        alfa_label = "Угол, град"
        omega_label = "Угловая скорость, град/с"
        eps_label = "Угловое ускорение, град/с²"
        zeta_label = "Рывок, град/с³"
    elif angle_unit == 'radian':
        alfa_series_loc = alfa_series
        omega_series_loc = omega_series
        eps_series_loc = eps_series
        zeta_series_loc = zeta_series
        alfa_target_loc = alfa_target
        omega_max_loc = omega_max
        omega_target_loc = omega_target
        eps_max_loc = eps_max
        eps_target_loc = eps_target
        alfa_label = "Угол, рад"
        omega_label = "Угловая скорость, рад/с"
        eps_label = "Угловое ускорение, рад/с²"
        zeta_label = "Рывок, рад/с³"
    # if angle unit is revolution, so time unit is minute
    elif angle_unit == 'revolution':
        alfa_series_loc = alfa_series/(2*np.pi)
        omega_series_loc = 60*omega_series/(2*np.pi)
        eps_series_loc = 3600*eps_series/(2*np.pi)
        zeta_series_loc = 216000*zeta_series/(2*np.pi)
        alfa_target_loc = alfa_target/(2*np.pi)
        omega_max_loc = 60*omega_max/(2*np.pi)
        omega_target_loc = 60*omega_target/(2*np.pi)
        eps_max_loc = 3600*eps_max/(2*np.pi)
        eps_target_loc = 3600*eps_target/(2*np.pi)
        alfa_label = "Угол, об"
        omega_label = "Угловая скорость, об/мин"
        eps_label = "Угловое ускорение, об/мин²"
        zeta_label = "Рывок, об/мин³"

    plt.figure(figsize=(10, 10))

    plt.subplot(4, 1, 1)
    plt.title("Координата")
    plt.plot(t_series[::plot_step], alfa_series_loc[::plot_step])
    plt.axhline(alfa_target_loc, linestyle="dashed", label="$α_{target}$")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel(alfa_label)

    plt.subplot(4, 1, 2)
    plt.title("Скорость")
    plt.plot(t_series[::plot_step], omega_series_loc[::plot_step])
    plt.axhline(omega_target_loc, linestyle="dashed", label="$ω_{target}$")
    plt.axhline(omega_max_loc, linestyle="dashdot", label="$ω_{max}$", color="C3")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel(omega_label)

    plt.subplot(4, 1, 3)
    plt.title("Ускорение")
    plt.plot(t_series[::plot_step], eps_series_loc[::plot_step])
    plt.axhline(eps_target_loc, linestyle="dashed", label="$ε_{target}$")
    plt.axhline(eps_max_loc, linestyle="dashdot", label="$ε_{max}$", color="C3")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel(eps_label)

    plt.subplot(4, 1, 4)
    plt.title("Рывок")
    plt.plot(t_series[::plot_step], zeta_series_loc[::plot_step])
    plt.xlabel("Время, с")
    plt.ylabel(zeta_label)

    plt.tight_layout()
    plt.show()

## Интерактивный подбор параметров и построение трапеции

In [63]:
alfa_target_input = widgets.BoundedFloatText(
    value=0.05,
    min=0.001,
    max=2.0,
    step=0.1,
    description="α_target, рад",
    tooltip="Целевой угол"
)
omega_max_input = widgets.BoundedFloatText(
    value=0.1,
    min=0.001,
    max=100,
    step=0.1,
    description="ω_max, рад/с",
    tooltip="Абсолютная максимальная скорость (реальная максимальная может быть меньше)"
)
eps_max_input = widgets.BoundedFloatText(
    value=4,
    min=0.01,
    max=1000,
    step=0.1,
    description="ε_max, рад/с²",
    tooltip="Абсолютное максимальное ускорение (реальное может быть меньше)"
)
zeta_input = widgets.BoundedFloatText(
    value=100000,
    min=0.001,
    max=10000000,
    step=0.1,
    description="ζ, рад/с³",
    tooltip="Рывок"
)

calculate_trapezoid_button = widgets.Button(
    description="Рассчитать профиль",
    tooltip="Целевая позиция"
)

output = widgets.Output()

def calculate_and_plot_trapezoid(b):    
    with output:
        output.clear_output()
        alfa_target = float(alfa_target_input.value)
        omega_max = float(omega_max_input.value)
        eps_max = float(eps_max_input.value)
        zeta = float(zeta_input.value)
        
        t_jerk, t_eps, t_omega, eps_target, omega_target = calcaulate_timings_and_targets(alfa_target, omega_max, eps_max, zeta)
        t_series, zeta_series, eps_series, omega_series, alfa_series = calculate_time_series(t_jerk, t_eps, t_omega, zeta)
        plot_trapezoid(
            t_series, zeta_series, eps_series, omega_series, alfa_series,
            eps_target, omega_target, alfa_target, omega_max, eps_max
        )
        

calculate_trapezoid_button.on_click(calculate_and_plot_trapezoid)

display(
    alfa_target_input,
    omega_max_input,
    eps_max_input,
    zeta_input,
    calculate_trapezoid_button,
    output
)

BoundedFloatText(value=0.05, description='α_target, рад', max=2.0, min=0.001, step=0.1)

BoundedFloatText(value=0.1, description='ω_max, рад/с', min=0.001, step=0.1)

BoundedFloatText(value=4.0, description='ε_max, рад/с²', max=1000.0, min=0.01, step=0.1)

BoundedFloatText(value=100000.0, description='ζ, рад/с³', max=10000000.0, min=0.001, step=0.1)

Button(description='Рассчитать профиль', style=ButtonStyle(), tooltip='Целевая позиция')

Output()

In [73]:
layout = widgets.Layout(width='400px')
style = {'description_width': '100px'}

alfa_target_input_deg = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=3.0,
    min=0.1,
    max=120.0,
    step=0.1,
    description="α_target, град",
    tooltip="Целевой угол"
)
omega_max_input_deg = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=6.0,
    min=0.01,
    max=100,
    step=0.1,
    description="ω_max, град/с",
    tooltip="Абсолютная максимальная скорость (реальная максимальная может быть меньше)"
)
eps_max_input_deg = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=230,
    min=0.01,
    max=1000,
    step=0.1,
    description="ε_max, град/с²",
    tooltip="Абсолютное максимальное ускорение (реальное может быть меньше)"
)
zeta_input_deg = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=6.0e6,
    min=100,
    max=100000000,
    step=0.1,
    description="ζ, град/с³",
    tooltip="Рывок"
)

calculate_trapezoid_button = widgets.Button(
    description="Рассчитать профиль",
    tooltip="Целевая позиция"
)

output = widgets.Output()

def calculate_and_plot_trapezoid(b):
    with output:
        output.clear_output()
        alfa_target = (np.pi/180)*float(alfa_target_input_deg.value)
        omega_max = (np.pi/180)*float(omega_max_input_deg.value)
        eps_max = (np.pi/180)*float(eps_max_input_deg.value)
        zeta = (np.pi/180)*float(zeta_input_deg.value)
        
        t_jerk, t_eps, t_omega, eps_target, omega_target = calcaulate_timings_and_targets(alfa_target, omega_max, eps_max, zeta, angle_unit='degree')
        t_series, zeta_series, eps_series, omega_series, alfa_series = calculate_time_series(t_jerk, t_eps, t_omega, zeta)
        plot_trapezoid(
            t_series, zeta_series, eps_series, omega_series, alfa_series,
            eps_target, omega_target, alfa_target, omega_max, eps_max,
            angle_unit='degree'
        )
        

calculate_trapezoid_button.on_click(calculate_and_plot_trapezoid)

display(
    alfa_target_input_deg,
    omega_max_input_deg,
    eps_max_input_deg,
    zeta_input_deg,
    calculate_trapezoid_button,
    output
)

BoundedFloatText(value=3.0, description='α_target, град', layout=Layout(width='400px'), max=120.0, min=0.1, st…

BoundedFloatText(value=6.0, description='ω_max, град/с', layout=Layout(width='400px'), min=0.01, step=0.1, sty…

BoundedFloatText(value=230.0, description='ε_max, град/с²', layout=Layout(width='400px'), max=1000.0, min=0.01…

BoundedFloatText(value=6000000.0, description='ζ, град/с³', layout=Layout(width='400px'), max=100000000.0, min…

Button(description='Рассчитать профиль', style=ButtonStyle(), tooltip='Целевая позиция')

Output()

In [67]:
layout = widgets.Layout(width='400px')
style = {'description_width': '100px'}
alfa_target_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=0.008,
    min=0.001,
    max=2.0,
    step=0.1,
    description="α_target, об",
    tooltip="Целевой угол"
)
omega_max_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=1.0,
    min=0.001,
    max=100,
    step=0.1,
    description="ω_max, об/мин",
    tooltip="Абсолютная максимальная скорость (реальная максимальная может быть меньше)"
)
eps_max_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=2300,
    min=1,
    max=100000,
    step=0.1,
    description="ε_max, об/мин²",
    tooltip="Абсолютное максимальное ускорение (реальное может быть меньше)"
)
zeta_input = widgets.BoundedFloatText(
    layout=layout,
    style=style,
    value=3.0e9,
    min=0.1,
    max=3.0e10,
    step=0.1,
    description="ζ, об/мин³",
    tooltip="Рывок"
)

calculate_trapezoid_button = widgets.Button(
    description="Рассчитать профиль",
    tooltip="Целевая позиция"
)

output = widgets.Output()

def calculate_and_plot_trapezoid(b):    
    with output:
        output.clear_output()
        alfa_target = (2*np.pi)*float(alfa_target_input.value)
        omega_max = (2*np.pi)*(omega_max_input.value)/60
        eps_max = (2*np.pi)*float(eps_max_input.value)/3600
        zeta = (2*np.pi)*float(zeta_input.value)/216000
        
        t_jerk, t_eps, t_omega, eps_target, omega_target = calcaulate_timings_and_targets(alfa_target, omega_max, eps_max, zeta, angle_unit='revolution')
        t_series, zeta_series, eps_series, omega_series, alfa_series = calculate_time_series(t_jerk, t_eps, t_omega, zeta)
        plot_trapezoid(
            t_series, zeta_series, eps_series, omega_series, alfa_series,
            eps_target, omega_target, alfa_target, omega_max, eps_max,
            angle_unit='revolution'
        )
        

calculate_trapezoid_button.on_click(calculate_and_plot_trapezoid)

display(
    alfa_target_input,
    omega_max_input,
    eps_max_input,
    zeta_input,
    calculate_trapezoid_button,
    output
)

BoundedFloatText(value=0.008, description='α_target, об', layout=Layout(width='400px'), max=2.0, min=0.001, st…

BoundedFloatText(value=1.0, description='ω_max, об/мин', layout=Layout(width='400px'), min=0.001, step=0.1, st…

BoundedFloatText(value=2300.0, description='ε_max, об/мин²', layout=Layout(width='400px'), max=100000.0, min=1…

BoundedFloatText(value=3000000000.0, description='ζ, об/мин³', layout=Layout(width='400px'), max=30000000000.0…

Button(description='Рассчитать профиль', style=ButtonStyle(), tooltip='Целевая позиция')

Output()